In [ ]:
!pip install optuna

     |████████████████████████████████| 204kB 6.3MB/s 
     |████████████████████████████████| 1.1MB 19.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 112kB 31.7MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=76403438a02b435ba9389f3ad2f27fce4f476346eaa37cd1b59e4872e2be9393
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-1.5.0-cp36-none-any.whl size=276145 sha256=1fd581ccd328903535531649e2cc9828ffa9718f3c19ec1e27cd4d307142d47f
  Stored in directory: /root/.cache/pip/wheels/38/2

In [ ]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import BayesianRidge, LinearRegression, Ridge

from lightgbm import LGBMRegressor

from xgboost import XGBClassifier

# from autofeat import AutoFeatRegressor

import optuna

In [ ]:
def accuracy(y_true,y_pred):
  return 100 * max(0,1-np.sqrt(mean_squared_error(y_true,y_pred)))

In [ ]:
path = 'drive/My Drive/Hackerearth/ML1/Dataset/'

In [ ]:
train = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')

In [ ]:
test_eid = test['Employee_ID'].copy()

In [ ]:
train['Employee_ID'] = train['Employee_ID'].apply(lambda x: int(x.split('_')[1]))
test['Employee_ID'] = test['Employee_ID'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
cols = train.columns[train.dtypes == 'object']

for col in cols:
  dummies = pd.get_dummies(train[col])
  train.drop([col],axis=1,inplace=True)
  train = pd.concat([train,dummies],axis=1)
  dummies = pd.get_dummies(test[col])
  test.drop([col],axis=1,inplace=True)
  test = pd.concat([test,dummies],axis=1)

In [ ]:
for col in train.columns:
  train[col].fillna(train[col].median(),inplace=True)

In [ ]:
funcs = [np.add, np.subtract, np.multiply, np.divide]

for i, func in enumerate(funcs):
    train['Age_TOS'+str(i+1)] = func(train['Age'],train['Time_of_service'])
    test['Age_TOS'+str(i+1)] = func(test['Age'],test['Time_of_service'])

    train['AT_GR'+str(i+1)] = func(train['Age_TOS'+str(i+1)], train['growth_rate'])
    test['AT_GR'+str(i+1)] = func(test['Age_TOS'+str(i+1)], test['growth_rate'])

    train['TS_GR'+str(i+1)] = func(train['Time_of_service'], train['growth_rate'])
    test['TS_GR'+str(i+1)] = func(test['Time_of_service'], test['growth_rate'])

    train['PS_WLB'+str(i+1)] = func(train['Pay_Scale'], train['Work_Life_balance'])
    test['PS_WLB'+str(i+1)] = func(test['Pay_Scale'], test['Work_Life_balance'])

    train['Age_TSP'+str(i+1)] = func(train['Age'], train['Time_since_promotion'])
    test['Age_TSP'+str(i+1)] = func(test['Age'],test['Time_since_promotion'])

    train['Age_EMP'+str(i+1)] = func(train['Age'], train['Employee_ID'])
    test['Age_EMP'+str(i+1)] = func(test['Age'], test['Employee_ID'])

    train['TOS_EMP'+str(i+1)] = func(train['Time_of_service'],train['Employee_ID'])
    test['TOS_EMP'+str(i+1)] = func(test['Time_of_service'],test['Employee_ID'])

    train['ATOS_EMP'+str(i+1)] = func(train['Age_TOS'+str(i+1)], train['Employee_ID'])
    test['ATOS_EMP'+str(i+1)] = func(test['Age_TOS'+str(i+1)], test['Employee_ID'])
    
for i,col in enumerate(test.columns):
    test[col+str(i+1)] = test[col]**2
    test[col+str(i+10)] = test[col]**3
    train[col+str(i+1)] = train[col]**2
    train[col+str(i+10)] = train[col]**3


var_names = ['VAR'+str(i) for i in range(1,8)]
sums_tr = []
sums_ts = []
for row in train[var_names].itertuples():
    sums_tr.append(row[1]+row[2]+row[3]+row[4]+row[5]+row[6]+row[7])
    
for row2 in test[var_names].itertuples():
    sums_ts.append(row2[1]+row2[2]+row2[3]+row2[4]+row2[5]+row2[6]+row2[7])
    
train['sum_VARS'] = sums_tr
test['sum_VARS'] = sums_ts

In [ ]:
X = train.drop(['Attrition_rate'],axis=1)
y = train['Attrition_rate'].copy().values

In [ ]:
X = X.replace([np.inf, -np.inf], np.nan)
test = test.replace([np.inf, -np.inf], np.nan)
for cols in X.columns:
  X[cols].fillna(X[cols].mean(),inplace=True)
  test[cols].fillna(X[cols].mean(),inplace=True)

In [ ]:
af = AutoFeatRegressor(feateng_steps = 2 , verbose=True)

NameError: ignored

In [ ]:
X_ = af.fit_transform(X, y)

[AutoFeat] The 2 step feature engineering process could generate up to 52003 features.
[AutoFeat] With 7000 data points this new feature matrix would use about 1.46 gb of space.
[feateng] Step 1: transformation of original features
[feateng] Generated 84 transformed features from 46 original features - done.
[feateng] Step 2: first combination of features
[feateng] Generated 8235 feature combinations from 8385 original feature tuples - done.
[feateng] Generated altogether 8415 new features in 2 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 6565 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 0 features after noise filtering
[featsel] WARNING: Not a single good features was found...
[AutoFeat] Final dataframe with 46 feature 

In [ ]:
for idd in test['Employee_ID']:
  if not train[train['Employee_ID']==idd].empty:
    print('here')

In [ ]:
test_ = af.transform(test.values)

NameError: ignored

In [ ]:
path = 'drive/My Drive/Hackerearth/ML1/'
X = pd.read_csv(path+'X_modified.csv')
test = pd.read_csv(path+'test_modified.csv')

In [ ]:
grid_lgbm = {
    'max_depth':16,
    'learning_rate':0.002,
    'n_estimators':1704,
    'min_child_weight':17,
    'eta':0.364
}
grid_bayesian = {
    'n_iter':10000,
    'tol':6.42e-06,
    'alpha_1':0.0001,
    'alpha_2':6.58e-05,
    'lambda_1':0.999,
    'lambda_2':2.24e-05,
    'compute_score':True,
    'fit_intercept':True,
    'normalize':False
}
grid_ridge = {
    'max_iter':90000,
    'alpha': 0.23143352151759083, 
    'fit_intercept': True, 
    'normalize': False, 
    'tol': 0.0005243805101392493, 
    'solver': 'saga'
}

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mean_squared_error

In [ ]:
def perc_Tunning(trial):
  p1 = trial.suggest_loguniform('p1',0.1,0.9)
  p2 = trial.suggest_loguniform('p2',0.1,0.9)
  p3 = trial.suggest_loguniform('p3',0.1,0.9)

  scores = []
  for tr_in, val_in in KFold().split(X, y):
    BR = BayesianRidge(**grid_bayesian)
    BR.fit(X.iloc[tr_in], y[tr_in])
    LR = Ridge(**grid_ridge)
    LR.fit(X.iloc[tr_in], y[tr_in])
    lgbm = LGBMRegressor(**grid_lgbm)
    lgbm.fit(X.iloc[tr_in], y[tr_in])
    scores.append(accuracy(y[val_in],BR.predict(X.iloc[val_in])*p1+LR.predict(X.iloc[val_in])*p2 + lgbm.predict(X.iloc[val_in])*p3))
 
  return -np.mean(scores)

In [ ]:
def hyp_Tunning(trial):
  d1 = trial.suggest_int('d1',2,2000)
  d2 = trial.suggest_int('d2',2,2000)
  d3 = trial.suggest_loguniform('d3',0.1,0.99)
  d4 = trial.suggest_int('d4',2,2000)
  lr = trial.suggest_loguniform('lr',1e-6,1)

  model = Sequential()
  model.add(Dense(d1,activation='relu',dtype='float32'))
  model.add(Dense(d2,activation='relu'))
  model.add(Dropout(d3))
  model.add(Dense(d4,activation='relu'))
  model.add(Dense(1))
  model.compile(optimizer = Adam(learning_rate=lr),loss = 'mae')
  
  scores = []
  for tr_in, val_in in KFold().split(X, y):
    model.fit(X.iloc[tr_in].values,y[tr_in],epochs=50,verbose=0)
    preds = model.predict(X.iloc[val_in].values)
    preds = preds.reshape((preds.shape[0],))
    scores.append(accuracy(y[val_in],preds))
 
  return -np.mean(scores)

In [ ]:
study = optuna.create_study()
study.optimize(hyp_Tunning,n_jobs=-1)

[I 2020-06-13 14:40:18,409] Finished trial#1 with value: -32.96061396598816 with parameters: {'d1': 1883, 'd2': 1781, 'd3': 0.5509652089537553, 'd4': 164, 'lr': 5.8538815643597755e-05}. Best is trial#1 with value: -32.96061396598816.
[I 2020-06-13 14:40:18,458] Finished trial#0 with value: -76.58621191978455 with parameters: {'d1': 1566, 'd2': 1176, 'd3': 0.35960408980706643, 'd4': 1586, 'lr': 0.1314378190977103}. Best is trial#0 with value: -76.58621191978455.
[I 2020-06-13 14:42:33,547] Finished trial#2 with value: -76.35322362184525 with parameters: {'d1': 671, 'd2': 1692, 'd3': 0.6159575164294089, 'd4': 29, 'lr': 2.1490280768334244e-06}. Best is trial#0 with value: -76.58621191978455.
[I 2020-06-13 14:42:33,757] Finished trial#3 with value: -12.789307832717896 with parameters: {'d1': 1462, 'd2': 156, 'd3': 0.3269682234607257, 'd4': 1199, 'lr': 0.9366494293725399}. Best is trial#0 with value: -76.58621191978455.
[I 2020-06-13 14:46:40,048] Finished trial#4 with value: -64.6413406729

In [ ]:
scores = []
for tr_in, val_in in KFold().split(X_, y):
  LR = LGBMRegressor()
  LR.fit(X_.iloc[tr_in], y[tr_in])
  scores.append(accuracy(y[val_in],LR.predict(X_.iloc[val_in])))

In [ ]:
np.mean(scores)

81.0431116674669

In [ ]:
LR = LGBMRegressor()
LR.fit(X_, y)
preds = LR.predict(test_)

In [ ]:
ss = pd.DataFrame()
ss['Employee_ID'] = test_eid
ss['Attrition_rate'] = preds

ss.to_csv(path+'submission.csv',index=False)